### Install dependencies

In [1]:
!pip install accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.1 MB/s eta 0:00:0000:0100:01


In [2]:
import os
os.system("pip install -t /tmp/submission/lib accelerate bitsandbytes")

  Using cached bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.0 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 4.7 MB/s eta 0:00:00a 0:00:01
Using cached bitsandbytes-0.43.2-py3-none-manylinux_2_24_x86_64.whl (137.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.3/417.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 54.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.6.1 requires cubinlinker, which is not installed.
cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.6.1 requires ptxcompiler, which is not installed.
cuml 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.6.1 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.9.0 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
ucx-py 0.38.0 requires libucx<1.16,>=1.15.0, which is not installed.
ucxx 0.38.0 requires libucx>=1.15.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10

0

### Download model snapshot (Qwen2-7B-Instruct)

In [3]:
from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

model_path = Path("/tmp/submission/")
if model_path.exists():
    shutil.rmtree(model_path)
model_path.mkdir(parents=True)

snapshot_download(
    repo_id= "Qwen/Qwen2-7B-Instruct",
    local_dir=model_path
)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/tmp/submission'

In [4]:
#%%writefile main.py

# Comment out the line above if performing an evaluation session. 

import os

#This checks to see what environment this script is running in. That way you can submit and evaluate in different environments.
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"

import sys
import itertools
import re
import torch
import typing as t
from pathlib import Path
from pydantic import BaseModel, Field, field_validator
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from IPython.display import display, Markdown

model_initialized = False
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t_dtype = torch.bfloat16
DEBUG = False

#==================QUANTIZATION CONFIG================
# You need to quantize Gwen 2 7b. It is too big to fit on the single T4 GPU used in the competition.
# This is included for reference so that you can see what the config looks like to quantize a larger model.
# We also deploy a conditional statement that checks if the model is already loaded. If it is, we skip loading to prevent OOM errors.
# This is super helpful when modifying the script, checking the evaluation session, and trying something new. 

if 'model' in locals() and 'tokenizer' in locals() and model is not None and tokenizer is not None:
    print("Model and tokenizer already exist. Skipping creation.")
else:
    quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                             bnb_4bit_compute_dtype=t_dtype)
    
    model = AutoModelForCausalLM.from_pretrained(KAGGLE_AGENT_PATH, 
                                                 device_map=device, 
                                                 torch_dtype=t_dtype,
                                                 quantization_config=quantization_config)
    
    tokenizer = AutoTokenizer.from_pretrained(KAGGLE_AGENT_PATH, 
                                              torch_dtype=t_dtype)

        
#==================OBSERVATION CLASS FOR KAGGLE ENVIRONMENT================
# This class takes the obs input from the kaggle environment for use in this script.
# Shout out to Team Rigging public notebook for the observation class.

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_text(self, *, skip_guesses: bool = False, skip_answer: bool = False, skip_question: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""{'**Question:** ' + question if not skip_question else ''} -> {'**Answer:** ' + answer if not skip_answer else ''}
            {'**Previous Guess:** ' + guess if not skip_guesses else ''}
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


# =================SYSTEM PROMPT================
# Here is your system prompt that is inserted into the message template during text generation.
# For instruction/chat models, this helps provide context and direction for the model to follow.
# There are many prompt strategies to deploy here that can be explored and researched. 
# You can also have different ssystem prompts for different roles, game states, or game phases (early/mid/late game).

system_prompt = """You are playing the interactive unknown entity identification game, 20 questions.
You are an experienced player of all roles, questioner, answerer, and guesser.
Your objective is to guess the secret keyword in as few turns as possible and win."""

# =================ASKER AGENT================
# The asker agent first asks is the keyword a place then asks if it is located north of the equator. No LLM is called here.
# If the answer to the place quesiton is no, it asks if it is a non-living thing.
# This helps provide a game environment few-shot example to the LLM AND helps bisect the first categories.
# Afterwards (steps > 1), it asks a generated question based on the previous history.
# You could add additional conditional questions before calling the LLM to further bisect the search space.
# You could also add text generation steps to initiate chain-of-thought reasoning, or other techniques.
# Make sure to check your max_new_tokens and temperature parameters.
# As your prompt complexity increases, you will want to create different generate_text functions, with different parameters.
# Like a generate_long with has much higher new token limit and higher temperature.

def ask(observation: Observation) -> str:
    if observation.step == 0:
        return "Is the secret keyword a place?"
    elif observation.step == 1:
        if len(observation.answers) > 0 and observation.answers[0] == "yes":
            return "Is it a city?"
        else:
            return "Is it a living thing?"
    game_history = observation.get_history_text(skip_guesses=True)

    try:
        think = generate_text(f"""
        You are playing as questioner and currently asking the next question. Here's what you know so far:

====== GAME HISTORY =====
{game_history}
=========================

        Based on the history above, explian which direction the questions should go next. 
        """)
        
        parse = generate_text(f"""<start_of_turn>model\n
        Review the current information below and give your next best question. 
        If you don't have much information, it is best to ask a yes/no question that eliminates categories from the search space.  

        Summary: "{think}""
        
        Just ask a question. Don't provide any more text than the question and a question mark. 
        """
        ).lower()
        if DEBUG:
            display(Markdown(f"### DEBUG: Questioner thinking:"))
            display(Markdown(think))
            display(Markdown(f"### DEBUG: Questioner parsing:"))
            display(Markdown(parse))
        return parse
    except Exception as e:
        print(f"Error while asking question: {e}")
        return 'Is it a place?'
    

# =================ANSWERER AGENT================
# Loop 5 times and take the most frequent response
# This was a technique used in the Team Rigging notebooks. 
# It is slightly more effective than a single response. 

def answer(observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("Keyword wasn't provided to answerer", file=sys.stderr)
        return "yes"
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            response = generate_text(f"""
            20 Questions game. Answer yes/no for this keyword: {observation.keyword} 

            Question: "{last_question}"
            
            Rules:
            1. Only consider your keyword: {observation.keyword} that is in the category: {observation.category}.
            2. Answer 'yes' or 'no. Nothing else. 
            Your answer here:
            """
            ).lower()
            if DEBUG:
                display(Markdown(f"### DEBUG: Answerer response:"))
                display(Markdown(f"DEBUG: {response}"))
            
            yes_no = re.findall(r'\b(yes|no)\b', response)
            if yes_no:
                responses.append(yes_no[0])

        if DEBUG:
            display(Markdown(f"DEBUG: All Answerer responses are {responses}"))
        return max(set(responses), key=responses.count)

    except Exception as e:
        print(f"Error in answerer: {e}", file=sys.stderr)
        return "yes"

# ================GUESSER AGENT================
# This agent uses the game history to make a guess.
# In some instances, this is not helpful because the LLM may get stuck in a associated word loop.
# Try different prompt strategies including setting skip guesses to True. 
# Parsing strategies helps make sure thet the game reponse is valid and understandable.


def guess(observation: Observation) -> str:
    game_history = observation.get_history_text(skip_guesses=False)
    
    think = generate_text(f"""
    You are currently the guesser, making an informed guess of the keyword. 

== GAME HISTORY ==
{game_history}
==================

    Suggest 1 category that is still relevant to the secret keyword based on this information and suggest a possible guess.
    Explain your reasoning. 
    """
    )
    
    parse = generate_text(f"""
    Review the response below and give your next best guess. 
    If you don't have much information, make a wild, but specific, guess.
    Don't ask the guess like a question, just give the word or phrase guess. 

    Summary: "{think}"

    Your guess here:
    """
    ).lower()
    if DEBUG:
        print(f"Game history: {game_history}")
        display(Markdown(f"### DEBUG: Guesser thinking:"))
        display(Markdown(think))
        display(Markdown(f"### DEBUG: Guesser parse:"))
        display(Markdown(parse))
    
    return parse

#==============LLM MESSAGE GENERATION===============
# There are few parameters you may want to modify.
# Temperature sets the randomness of the generated text. 
# Lower values are more deterministic, higher - more creative. 
# max_new_tokens sets the length of the generated text.

    
def generate_text(prompt:str) -> str:
    sys_prompt = system_prompt
    messages = [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": prompt},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True)
    
    inputs = tokenizer([text], return_tensors="pt").to(device)
    
    generated_ids = model.generate(inputs.input_ids,
                                   max_new_tokens=350, 
                                   do_sample=True, 
                                   temperature=0.1) #this sets the randomness/creativity of the generated text, lower is more deterministic
    
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response

#================AGENT INITIALIZATION FOR KAGGLE ENV================
# It may be fun to change one of the agents to 'input' to play against the model during eval!
# Bonus, this opens up a possibility to create a fine-tuning dataset! How neat!
# Maybe you could alter the eval session to be a fine-tuning dataset building session that saves the output in a dataframe?
# The possibilities are endless but time is limited.  

agent = None

def observe(obs: t.Any) -> str:
    global agent
    observation = Observation(**obs.__dict__)

    try:
        match observation.turnType:
            case "ask":
                agent = ask(observation)
                return agent
            case "answer":
                agent = answer(observation)
                return agent
            case "guess":
                agent = guess(observation)
                return agent

            case _:
                raise ValueError("Unknown turn type")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    return observe(obs)

# =========================ENABLE TORCH BACKEND===========================
# Sometimes you'll get errors when running the script in this notebook. 
# These backend settings ensure that the script runs without errors. Yay!

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Submission

In [ ]:
!apt install pigz pv

### Evaluation

In [5]:
perform_eval = True

In [6]:
if perform_eval:
    import requests
    import json
    import re
    import typing as t
    import random
    import time
    from IPython.display import display, Markdown
    import signal

    # ===============FETCH LATEST KEYWORDS FROM KAGGLE GITHUB================
    # This does not train the model to look for the keywords.
    # This just fetches the list so it can be used in the evaluation.
    
    url = "https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py"
    response = requests.get(url)

    if response.status_code == 200:
        match = re.search(r'KEYWORDS_JSON = """(.*?)"""', response.text, re.DOTALL)
        if match:
            json_str = match.group(1)
            keywords_dict = json.loads(json_str)
        else:
            print("Could not find the KEYWORDS_JSON variable in the file.")
    else:
        print("Request failed with status code:", response.status_code)
        import time

    def select_random_keyword(keywords_dict: t.List[dict]) -> str:
        category = random.choice(keywords_dict)
        keyword_dict = random.choice(category['words'])
        return keyword_dict['keyword'].lower()
    
    #===============20 QUESTIONS EVALUATION SESSION=====================
    def input_timeout(prompt, timeout):
        def timeout_handler(signum, frame):
            raise TimeoutError("Input timed out after {} seconds".format(timeout))

        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout)

        try:
            user_input = input(prompt)
            signal.alarm(0)
            return user_input
        except TimeoutError as e:
            print(e)
            return None

    timeout = 10

    try:
        DEBUG_input = input_timeout("Enable verbose debug mode? (y/n) [default is n] ", timeout)
        DEBUG = DEBUG_input.lower() == 'y' if DEBUG_input else False
    except:
        DEBUG = False
        print("No input received, defaulting to false.")

    class MockObservation:
        def __init__(self, step: int, role: str, turnType: str, keyword: str, category: str, questions: list[str], answers: list[str], guesses: list[str]):
            self.step = step
            self.role = role
            self.turnType = turnType
            self.keyword = keyword
            self.category = category
            self.questions = questions
            self.answers = answers
            self.guesses = guesses

    def test_20_questions():
        global DEBUG
        step = 0
        role = "answerer"
        turnType = "ask"
        keyword = select_random_keyword(keywords_dict)
        category = ""
        questions = []
        answers = []
        guesses = []
        display(Markdown("# Starting 20 questions eval game..."))
        display(Markdown(f"### **Keyword:** {keyword}"))

        for i in range(60):
            obs = MockObservation(step, role, turnType, keyword, category, questions, answers, guesses)

            start_time = time.time()
            response = agent_fn(obs, None)
            end_time = time.time()

            response_time = end_time - start_time
            if response_time > 60:
                display(Markdown(f"**WARNING:** Response time too long and may be disqualified from the game: {response_time:.2f} sec. Make sure you have GPU acceleration enabled in the session options on the right side panel."))
                break

            # Record the response in the appropriate list
            if turnType == 'ask':
                questions.append(response)
                turnType = 'answer'
            elif turnType == 'answer':
                answers.append(response)
                turnType = 'guess'
            elif turnType == 'guess':
                guesses.append(response)
                if response.lower() == keyword.lower():
                    display(Markdown(f"## **Keyword '{keyword}' guessed correctly! Ending game.**"))
                    break
                turnType = 'ask'
                step += 1

            display(Markdown(f"Step {step} | Response: {response} | {response_time:.2f} sec"))
        display(Markdown(f"Final Questions: {', '.join(questions)}"))
        display(Markdown(f"Final Answers: {', '.join(answers)}"))
        display(Markdown(f"Final Guesses: {', '.join(guesses)}"))

    # Run the test
    test_20_questions()

Enable verbose debug mode? (y/n) [default is n]  y


# Starting 20 questions eval game...

### **Keyword:** iraq

Step 0 | Response: Is the secret keyword a place? | 0.00 sec

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
2024-07-29 21:04:14.630849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 21:04:14.630959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 21:04:14.787881: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

greece

Step 20 | Response: greece | 21.79 sec

Final Questions: Is the secret keyword a place?, Is it a city?, is it a country?, is it located in europe?, is this country located in asia?, is the country relatively large in geographical size?, is the country's economy primarily based on agriculture?, does the country have a population exceeding 10 million people?, is the country's economy diversified or focused on industries other than agriculture?, does the country have a significant coastline or is it predominantly inland?, is the country's economy heavily reliant on manufacturing or industry?, is the country known for having a capital city that serves as a major economic hub?, is the country known for having a monarchy or a republic system of government?, does the country have a specific type of political system other than monarchy or republic?, "does the country have a significant presence due to tourism related to historical sites or natural wonders?", does the country belong to a particular subregion of asia?, is this country famous for hosting a specific annual event or festival?, "is the country predominantly known for a specific climate type?", does the country have a unique cultural tradition or practice that is widely recognized globally?, is the country located near the eastern coast of the subregion it belongs to?

Final Answers: yes, no, yes, no, yes, no, no, yes, no, no, no, yes, no, no, yes, yes, no, no, yes, yes

Final Guesses: **mountains**, japan, alps, south america, china, malaysia, germany, tokyo, japan, manufacturing, switzerland, singapore, singapore, germany, italy, italy, turkey, india, india, greece